In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D,BatchNormalization,Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
train_data_dir = '../input/whitebloodcell/whitebloodcell/train'
validation_data_dir = '../input/whitebloodcell/whitebloodcell/val'
test_Data_dir='../input/whitebloodcell/whitebloodcell/test'

num_classes =10
batch_size = 16
img_rows, img_cols = 224, 224

train_datagen = ImageDataGenerator()

validation_datagen = ImageDataGenerator()

test_datagen = ImageDataGenerator()

In [ ]:
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_rows, img_cols), batch_size=batch_size, class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory( validation_data_dir, target_size=(img_rows, img_cols), batch_size=batch_size, class_mode='categorical')
test_generator=test_datagen.flow_from_directory( test_Data_dir, target_size=(img_rows, img_cols), batch_size=batch_size, class_mode='categorical')

In [ ]:
def modelfunction(pretrained,weight):
    for layer in pretrained.layers:
        layer.trainable = False
    def addTopModel(bottom_model, num_classes):
        top_model = bottom_model.output
        top_model = Flatten()(top_model)
        top_model = Dense(num_classes, activation='softmax')(top_model)
        return top_model
    FC_Head = addTopModel(pretrained, num_classes)
    model = Model(inputs=pretrained.input, outputs=FC_Head)
    
    checkpoint = ModelCheckpoint(weight, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=5, verbose=1, factor=0.2, min_lr=0.0002)
    callbacks = [earlystop, checkpoint, learning_rate_reduction]
    
    metrics = ['accuracy', tf.keras.metrics.AUC(), tfa.metrics.CohenKappa(num_classes = 10), tfa.metrics.F1Score(num_classes = 10), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=metrics)
    
    batch_size = 32
    history = model.fit_generator(train_generator, steps_per_epoch=4716  // batch_size, epochs=50, callbacks=callbacks, validation_data=validation_generator, 
                              validation_steps=1571 // batch_size)
    
    #plt.figure(figsize=(10,5))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # "Loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    model.evaluate(test_generator)

# MobileNet

In [ ]:
from tensorflow.keras.applications import MobileNet
MobileNet = MobileNet(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))
modelfunction(MobileNet,'MobileNet.h5')

# VGG19

In [ ]:
from tensorflow.keras.applications import VGG19
vgg = VGG19(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))
modelfunction(vgg,'vgg.h5')

# Xception

In [ ]:
from tensorflow.keras.applications import Xception
xception = Xception(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))
modelfunction(xception,'xception.h5')

# Inception V3

In [ ]:
from tensorflow.keras.applications import InceptionV3
Inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))
modelfunction(Inception,'Inception.h5')

# InceptionResNetV2

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
InceptionResNet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))
modelfunction(InceptionResNet,'InceptionResNet.h5')

# DenseNet121

In [ ]:
from tensorflow.keras.applications import DenseNet121
DenseNet = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))
modelfunction(DenseNet,'DenseNet.h5')